In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os , re

In [2]:
train_txt = []
test_txt = []
for line in open(r"movie_data/full_train.txt",'r',encoding='utf-8'):
    train_txt.append(line.strip())
    
for line in open(r"movie_data/full_test.txt",'r',encoding='utf-8'):
    test_txt.append(line.strip())

In [3]:
train_txt[5]

"This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br />

In [9]:
no_space = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
rep1 = ""
rep2 = " "
def preprocess(txt):
    txt = [no_space.sub(rep1,line.lower()) for line in txt]
    txt = [space.sub(rep2,line.lower()) for line in txt]
    return txt
txt1 = preprocess(train_txt)
txt2 = preprocess(test_txt)

In [10]:
txt1[5]

'this isnt the comedic robin williams nor is it the quirky insane robin williams of recent thriller fame this is a hybrid of the classic drama without over dramatization mixed with robins new love of the thriller but this isnt a thriller per se this is more a mystery suspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until williams character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all its worth a watch though its definitely not friday saturday night fare it rates a   from the fiend '

In [11]:
cv = CountVectorizer(binary=True)
x = cv.fit_transform(txt1)
x_test = cv.transform(txt2)

In [12]:
len(train_txt)

25000

In [13]:
len(test_txt)

25000

In [15]:
target = [1 if i < 12500 else 0 for i in range(25000)]
x_train,x_val,y_train,y_val = train_test_split(x,target,train_size=0.75)
lg = LogisticRegression(C=0.05)
lg.fit(x_train,y_train)
print ("Accuracy: %s"% accuracy_score(y_val, lg.predict(x_val)))

Accuracy: 0.88272


In [16]:
#testing the model
pred = lg.predict(x_test)
print("Testing Accuracy: %s"%accuracy_score(target,pred))

Testing Accuracy: 0.87736


In [18]:
len(cv.get_feature_names_out())

90860

In [21]:
lg.coef_

array([[ 0.00217421,  0.04067065,  0.04095442, ...,  0.        ,
        -0.02582428,  0.        ]])

In [20]:
len(lg.coef_[0])

90860

In [36]:
feat_coef = {word: coef for word, coef in zip(cv.get_feature_names_out(),lg.coef_[0])}
list(feat_coef.items())[:10]

[('____', 0.002174208986482434),
 ('_____', 0.040670646368839214),
 ('______', 0.04095441603911098),
 ('_________', 0.020343455383756908),
 ('____________________________________', 0.0012403489512946737),
 ('_____________________________________', 0.0016680047112203847),
 ('__________________________________________________________________',
  0.002417711716843723),
 ('____insert', -0.00014645247404851622),
 ('_a', -0.0011179064655898031),
 ('_absolute', -0.0002773147637089581)]

In [37]:
for p in sorted(feat_coef.items(),key=lambda x: x[1],reverse=True)[:5]:
    print(p)

('excellent', 0.8681859530472555)
('perfect', 0.7861310327384623)
('great', 0.6779395373409677)
('best', 0.5568852601563656)
('superb', 0.5562969167207951)


In [38]:
for p in sorted(feat_coef.items(),key=lambda x: x[1])[:5]:
    print(p)

('worst', -1.3071219095225852)
('waste', -1.0889922646271815)
('awful', -1.0202642807458033)
('boring', -0.7846388556537885)
('poorly', -0.7204146634804687)


In [39]:
with open("SA Dict.txt",'w',encoding='utf-8') as f:
    for w,v in feat_coef.items():
        f.write(f"{w},{v}\n")